In [3]:
#External functions
%run draw_funcs.py
%run Kepler.py
#import modules
import sys, os, platform
import APC as APC
import plotly.graph_objects as go
import numpy as np
from IPython.display import display, Math, Latex

if platform.system()=="Windows":
    #avert render bug in windows 10
    import plotly.offline as pyo
    pyo.init_notebook_mode()

In [4]:
APC.Benchmark1000(8)

(8, 6.125343292)

In [5]:
def out2breaks(output):
    Ts = output.getTimes()
    Xs = output.getPosition()
    Vs = output.getVelocity()
    #Segment orbits into individual revolutions of the Earth
    prev_y=0.0
    orbit_breaks =[0]
    for i,y in enumerate(Xs[1]):
        if prev_y<0 and y>0:
            orbit_breaks.append(i)
        prev_y = y
    orbits_x = []
    orbits_y = []
    orbits_t = []
    for i,start in enumerate(orbit_breaks[0:-1]):
        if not start==0:
            start=start-1
        end = orbit_breaks[i+1]+1
        orbits_x.append(Xs[0][start:end])
        orbits_y.append(Xs[1][start:end])
        orbits_t.append(Ts[start:end])

    #find orbital elements at each time step
    elms = np.zeros((len(orbit_breaks),10))
    ts = np.zeros(len(orbit_breaks))
    for i,ind in enumerate(orbit_breaks):
        x = np.array([Xs[0][ind],Xs[1][ind],Xs[2][ind]])
        v = np.array([Vs[0][ind],Vs[1][ind],Vs[2][ind]])
        elms[i,:] = rv2elm(x,v,muEarth)
        #[p, a, e, i, Om, w, f, E, M, s]

    return [orbits_x, orbits_y, orbits_t, elms] 

In [6]:
#LEO high eccentricity
q = 8000
e = .05
a = q/(1-e)
i = 63.435*np.pi/180
r0,v0 = elms2rv(a,e,i,0,0,0,muEarth)
T = 2*np.pi*np.sqrt(a**3/muEarth)
t0 = 0.0
tf = 10*T
#LEO
# r0 = [6500, 0.0, 0.0];                                    # Initial Position (km)
# v0 = (0, 7.8309, 0.0);                              # Initial Velocity (km/s)
# t0 = 0.0;                                                 # Initial Times (s)
# T = np.pi*np.sqrt(6500*3/muEarth)
# tf = 100*T;                                     # Final Time (s)
# #MEO
# r0 = [9000.0, 0.0, 0.0];                                    # Initial Position (km)
# v0 = [0.0, 6.7419845635570, 1.806509319188210];             # Initial Velocity (km/s)
# t0 = 0.0;                                                   # Initial Times (s)
# tf = 10*9.952014050491189e+03;                              # Final Time (s)

#sat props
mass = 1000;
area = 10;
reflectance = 1.5;
Cd = 2.0;

# #run APC code in single orbit mode with and without perturbations
output = APC.SinglePropagate(r0,v0,t0,tf,area,reflectance,mass,Cd,False,False,False)
dragoutput = APC.SinglePropagate(r0,v0,t0,tf,area,reflectance,mass,Cd,True,True,True)

print(r0)
print(v0)
print(T)

[8000.    0.    0.]
[-0.          3.23468895  6.46939235]
7690.610095766187


In [5]:
#Run parallel propagations
# statelist = APC.GenSigma13(r0,v0,10,.1)
# Orbits = APC.ParallelPropagate(statelist,t0,tf,area,reflectance,mass,Cd,True,True,True)

In [ ]:
fig = plotOrbit(output)
fig.show()
camera = dict(
    up=dict(x=0, y=0, z=1),
    eye=dict(x=.08, y=-.1, z=.05),
    center=dict(x=0,y=0,z=0)
)
diff = 6378
x0 = 8000
y0 = 0
z0 = 0
fig.update_layout(
    scene=dict(
    aspectmode = "cube",
    xaxis=dict(range=[x0-diff,x0+diff]),
    yaxis=dict(range=[y0-diff,y0+diff]),
    zaxis=dict(range=[z0-diff,z0+diff])),
    scene_camera=camera, 
    title="Initial Position Close-up")
fig.show()


In [ ]:
orbits_x, orbits_y,orbits_t, elms = out2breaks(output)
drag_x, drag_y,drag_t, dragelms = out2breaks(dragoutput)

In [ ]:
"""Plot topdown view"""
trace2d = go.Scatter(x=output.getPositionX(),y=output.getPositionY(),mode = 'lines')
trace2 = plot_circle(6378)
fig2d = go.Figure(data=[trace2d,trace2])
fig2d.layout.xaxis.range=(-42500,8500)
fig2d.layout.yaxis.range=(-20000,20000)
fig2d.update_layout(
    autosize = False,
    width = 600,
    showlegend=False,
    xaxis_constrain = 'domain',
    yaxis_scaleanchor= 'x',
    margin=dict(l=10, r=10, t=10, b=10))
fig2d.show()


In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
import plotly

from ipywidgets import interactive, HBox, VBox, widgets, interact

py.init_notebook_mode()
Xs = output.getPosition()
clamp = 10000
# load fig
fig = fig2d
# create FigureWidget from fig
f = go.FigureWidget(data=fig.data, layout=fig.layout)
f.add_trace(plot_circle(6378,opacity=1))
xmin = max(min(min(Xs[0]),-8000),-clamp)
ymin = max(min(min(Xs[1]),-8000),-clamp)
ymax = min(max(max(Xs[1]),8000),clamp)
xmax = min(max(max(Xs[0]),8000),clamp)
f.layout.yaxis.range=[ymin,ymax]
f.layout.xaxis.range=[xmin,xmax]
f.layout.xaxis.title="X (km)"
f.layout.yaxis.title="Y (km)"
f.layout.title = "77 Deg Gravity Polar View Nodal Precession t={:.2f} hours<br><sup>a={}, e={}, i={}</sup>".format(0,a,e,180*i/np.pi)
f.update_layout(title_x=0.5)
f.layout.margin=dict(l=20,r=20,t=60,b=20)
f.layout.height = 800
f.layout.width = 800

slider = widgets.IntSlider(
    min=0,
    max=len(orbits_y)-1,
    step=1,
    readout=True,
    description='Orbit')
slider.layout.width = '800px'

# our function that will modify the xaxis range
def update_range(y):
    f.plotly_restyle({'x':[orbits_x[y]],'y':[orbits_y[y]]},0)
    f.layout.yaxis.range=[ymin,ymax]
    f.layout.xaxis.range=[xmin,xmax]
    title = "77 Deg Gravity Polar View Nodal Precession t={:.2f} hours<br><sup>a={:.2f} km, e={:.3f}, i={:.2f}°</sup>".format(orbits_t[y][0]/3600,a,e,i*180/np.pi)
    f.update_layout(title_text = title, title_x=0.5)
    f.show
    
vb = VBox((f, interactive(update_range,y=slider)))
vb.layout.align_items = 'flex-start'
vb

In [ ]:
save_image=False
if not os.path.exists("images"):
        os.mkdir("images")

for k in np.arange(0,len(orbits_y)):
    update_range(k)
    if save_image:
        f.write_image('images/fig{}.png'.format(k))